In [1]:
import requests
import os
from bs4 import BeautifulSoup
from bs2json import bs2json
import pandas as pd 
from selenium import webdriver
import scraper

In [6]:
id_link_df = pd.read_csv('csv/players_link.csv')
id_link_df.head()

,Unnamed: 0,sofifa_id,link
0,0,20801,https://fbref.com/en/players/dea698d9/all_comp...
1,1,158023,https://fbref.com/en/players/d70ce98e/all_comp...
2,2,190871,https://fbref.com/en/players/69384e5d/all_comp...
3,3,193080,https://fbref.com/en/players/7ba6d84e/all_comp...
4,4,192985,https://fbref.com/en/players/e46012d4/all_comp...


In [7]:
players_data_df = pd.read_csv('csv/players_data.csv')
players_data_df.head()

,Unnamed: 0,sofifa_id,short_name,long_name,age,nationality,club,team_position
0,0,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,33,Portugal,Juventus,LW
1,1,158023,L. Messi,Lionel Andrés Messi Cuccittini,31,Argentina,FC Barcelona,RW
2,2,190871,Neymar Jr,Neymar da Silva Santos Junior,26,Brazil,Paris Saint-Germain,CAM
3,3,193080,De Gea,David De Gea Quintana,27,Spain,Manchester United,GK
4,4,192985,K. De Bruyne,Kevin De Bruyne,27,Belgium,Manchester City,RCM


In [37]:
link_pos = id_link_df.join(players_data_df.set_index('sofifa_id'), on='sofifa_id', how='inner', rsuffix='--')[['sofifa_id', 'link', 'team_position']]

# Field Player scraper

In [36]:
field_players = link_pos[link_pos['team_position'] != 'GK']
field_players.head()

,sofifa_id,link,team_position
0,20801,https://fbref.com/en/players/dea698d9/all_comp...,LW
1,158023,https://fbref.com/en/players/d70ce98e/all_comp...,RW
2,190871,https://fbref.com/en/players/69384e5d/all_comp...,CAM
4,192985,https://fbref.com/en/players/e46012d4/all_comp...,RCM
5,155862,https://fbref.com/en/players/08511d65/all_comp...,LCB


In [48]:
# Dataframes for appending the values
shooting_df = pd.DataFrame(columns=["sofifa_id", "shots_on_target_pct", "goals", "shots_on_target", "shots_total", "average_shot_distance", "shots_free_kicks", "pens_made", "pens_att"])

passing_df = pd.DataFrame(columns=["sofifa_id", "passes_pct", "passes_completed", "passes_total_distance", "passes_progressive_distance", "passes_pct_short", "passes_pct_medium", "passes_pct_long" ,"assists", "assisted_shots", "passes_into_final_third", "passes_into_penalty_area", "crosses_into_penalty_area", "progressive_passes"])

defending_df = pd.DataFrame(columns=["sofifa_id", "tackles", "tackles_won", "dribble_tackles_pct", "pressure_regain_pct", "blocks", "interceptions", "clearances", "errors"])

dribbling_df = pd.DataFrame(columns=["sofifa_id", "dribbles_completed_pct", "players_dribbled_past", "carries", "carry_distance", "carry_progressive_distance", "miscontrols", "dispossessed"])

others_df = pd.DataFrame(columns=["sofifa_id", "points_per_match", "plus_minus", "plus_minus_wowy"])

miscel_df = pd.DataFrame(columns=["sofifa_id", "cards_yellow", "cards_red", "cards_yellow_red", "fouls", "fouled", "offsides", "crosses", "pens_won", "pens_conceded", "own_goals", "ball_recoveries", "aerials_won_pct"])

In [73]:
for index, row in field_players.iterrows():
    data = requests.get(row['link'])

    try:
        shooting_df = shooting_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'shooting_df'), index=shooting_df.columns), ignore_index=True)
        passing_df = passing_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'passing_df'), index=passing_df.columns), ignore_index=True)
        defending_df = defending_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'defending_df'), index=defending_df.columns), ignore_index=True)
        dribbling_df = dribbling_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'dribbling_df'), index=dribbling_df.columns), ignore_index=True)
        others_df = others_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'others_df'), index=others_df.columns), ignore_index=True)
        miscel_df = miscel_df.append(pd.Series(scraper.fplayer_scrap(row['sofifa_id'], data, 'miscel_df'), index=miscel_df.columns), ignore_index=True)
    except:
        print(f"Non valid data for player with ID {str(row['sofifa_id'])}")
         

Non valid data for player with ID 41236
Non valid data for player with ID 187961


In [79]:
shooting_df.to_csv('csv/fplayer_scraps/shooting.csv')
passing_df.to_csv('csv/fplayer_scraps/passing.csv')
defending_df.to_csv('csv/fplayer_scraps/defending.csv')
dribbling_df.to_csv('csv/fplayer_scraps/dribbling.csv')
others_df.to_csv('csv/fplayer_scraps/others.csv')
miscel_df.to_csv('csv/fplayer_scraps/miscel.csv')

# Goalkeeper scraper

In [38]:
gkeepers = link_pos[link_pos['team_position'] == 'GK']
gkeepers.head()

,sofifa_id,link,team_position
3,193080,https://fbref.com/en/players/7ba6d84e/all_comp...,GK
9,200389,https://fbref.com/en/players/ee8508c0/all_comp...,GK
10,192119,https://fbref.com/en/players/1840e36d/all_comp...,GK
11,167495,https://fbref.com/en/players/8778c910/all_comp...,GK
16,192448,https://fbref.com/en/players/6f51e382/all_comp...,GK


In [44]:
# Dataframes for appending the values
goalkeeping_df = pd.DataFrame(columns=["sofifa_id", "save_pct", "clean_sheets_pct", "pens_att_gk", "pens_saved"])

adv_gkeeping_df = pd.DataFrame(columns=["sofifa_id", "goals_against_gk", "pens_allowed", "free_kick_goals_against_gk", "corner_kick_goals_against_gk", "own_goals_against_gk", "passes_pct_launched_gk", "pct_passes_launched_gk", "passes_length_avg_gk", "pct_goal_kicks_launched", "goal_kick_length_avg", "crosses_stopped_pct_gk", "def_actions_outside_pen_area_gk", "def_actions_outside_pen_area_per90_gk", "avg_distance_def_actions_gk"])

passing_df = pd.DataFrame(columns=["sofifa_id", "passes_pct", "passes_total_distance", "passes_progressive_distance", "passes_pct_short", "passes_pct_medium", "passes_pct_long", "assists", "assisted_shots", "passes_into_final_third"])

goal_shoot_c_df = pd.DataFrame(columns=["sofifa_id", "sca", "sca_passes_live", "sca_passes_dead", "gca", "gca_passes_live", "gca_passes_dead"])

defensive_df = pd.DataFrame(columns=["sofifa_id", "tackles", "tackles_won", "dribble_tackles_pct", "pressure_regain_pct", "blocks", "blocked_shots", "blocked_shots_saves", "blocked_passes", "interceptions", "tackles_interceptions", "clearances", "errors"])

posession_df = pd.DataFrame(columns=["sofifa_id", "touches", "touches_live_ball", "dribbles_completed_pct", "carries", "carry_distance", "passes_received_pct", "miscontrols", "dispossessed"])

playtime_df = pd.DataFrame(columns=["sofifa_id", "points_per_match", "plus_minus"])

misc_df = pd.DataFrame(columns=["sofifa_id", "cards_yellow", "cards_red", "cards_yellow_red", "fouls", "fouled", "pens_conceded", "own_goals", "ball_recoveries", "aerials_won_pct"])

In [42]:
op = webdriver.ChromeOptions()
op.add_argument('headless')

browser = webdriver.Chrome('/Users/eduardooportoalonso/Documents/Cursos/Ironhack/datamad1020/ironhack-projects/pipelines-project/chromedriver', options=op)

In [45]:
for index, row in gkeepers.iterrows():
    browser.get(row['link'])
    
    goalkeeping_df = goalkeeping_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'goalkeeping_df'), index=goalkeeping_df.columns), ignore_index=True)
    adv_gkeeping_df = adv_gkeeping_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'adv_gkeeping_df'), index=adv_gkeeping_df.columns), ignore_index=True)
    passing_df = passing_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'passing_df'), index=passing_df.columns), ignore_index=True)
    goal_shoot_c_df = goal_shoot_c_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'goal_shoot_c_df'), index=goal_shoot_c_df.columns), ignore_index=True)
    defensive_df = defensive_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'defensive_df'), index=defensive_df.columns), ignore_index=True)
    posession_df = posession_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'posession_df'), index=posession_df.columns), ignore_index=True)
    playtime_df = playtime_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'playtime_df'), index=playtime_df.columns), ignore_index=True)
    misc_df = misc_df.append(pd.Series(scraper.gkeeper_scrap(row['sofifa_id'], browser, 'misc_df'), index=misc_df.columns), ignore_index=True)


In [47]:
goalkeeping_df.to_csv('csv/gkeeprs_scraps/goalkeeping.csv')
adv_gkeeping_df.to_csv('csv/gkeeprs_scraps/adv_gkeeping.csv')
passing_df.to_csv('csv/gkeeprs_scraps/passing.csv')
goal_shoot_c_df.to_csv('csv/gkeeprs_scraps/goal_shoot_c.csv')
defensive_df.to_csv('csv/gkeeprs_scraps/defensive.csv')
posession_df.to_csv('csv/gkeeprs_scraps/posession.csv')
playtime_df.to_csv('csv/gkeeprs_scraps/playtime.csv')
misc_df.to_csv('csv/gkeeprs_scraps/misc.csv')